<a href="https://colab.research.google.com/github/shreyanshsatvik/torch/blob/main/T5_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers -q

     |████████████████████████████████| 1.1MB 7.7MB/s 
     |████████████████████████████████| 3.0MB 28.7MB/s 
     |████████████████████████████████| 1.1MB 44.7MB/s 
     |████████████████████████████████| 890kB 53.1MB/s 


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = pd.read_csv('/content/drive/My Drive/news_summary/news_summary.csv',encoding = 'latin-1')

In [8]:
df.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [9]:
df =df[['text','ctext']]

In [10]:
df.head()

,text,ctext
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [11]:
df.text[0]

'The Administration of Union Territory Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7. The administration was forced to withdraw the decision within 24 hours of issuing the circular after it received flak from employees and was slammed on social media.'

In [12]:
df.ctext[0]

'The Daman and Diu administration on Wednesday withdrew a circular that asked women staff to tie rakhis on male colleagues after the order triggered a backlash from employees and was ripped apart on social media.The union territory?s administration was forced to retreat within 24 hours of issuing the circular that made it compulsory for its staff to celebrate Rakshabandhan at workplace.?It has been decided to celebrate the festival of Rakshabandhan on August 7. In this connection, all offices/ departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall tie rakhis to their colleagues,? the order, issued on August 1 by Gurpreet Singh, deputy secretary (personnel), had said.To ensure that no one skipped office, an attendance report was to be sent to the government the next evening.The two notifications ? one mandating the celebration of Rakshabandhan (left) and the other withdrawing the mandate (right) ? were issued by the Dama

In [13]:
df.ctext = 'summarize:' + df.ctext

In [14]:
df.head()

,text,ctext
0,The Administration of Union Territory Daman an...,summarize:The Daman and Diu administration on ...
1,Malaika Arora slammed an Instagram user who tr...,summarize:From her special numbers to TV?appea...
2,The Indira Gandhi Institute of Medical Science...,summarize:The Indira Gandhi Institute of Medic...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,summarize:Lashkar-e-Taiba's Kashmir commander ...
4,Hotels in Maharashtra will train their staff t...,summarize:Hotels in Mumbai and other Indian ci...


In [15]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [16]:
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state = 42)
val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(val_dataset.shape))

FULL Dataset: (4514, 2)
TRAIN Dataset: (3611, 2)
TEST Dataset: (903, 2)


In [17]:
train_dataset.head()

,text,ctext
0,"All restaurants, including five-star hotels, i...",summarize:Come April and you won?t have to go ...
1,The Chinese government has banned singer Justi...,"summarize:?I just need one more shot, second c..."
2,Pakistan on Saturday accused India of targetin...,summarize:Pakistan?s top military officer on S...
3,A 40-year-old woman in Hyderabad was burnt ali...,"summarize:In a tragic incident, a woman was bu..."
4,The Delhi Police has arrested a man working as...,summarize:A man who allegedly threatened to bl...


In [22]:
val_dataset.head()

,text,ctext
0,Hotels in Maharashtra will train their staff t...,summarize:Hotels in Mumbai and other Indian ci...
1,The Congress party has opened a bank called 'S...,"summarize:It sounds like satire, but make no m..."
2,"Tanveer Hussain, a 24-year-old Indian athlete ...",summarize:A 24-year-old Indian athlete has bee...
3,"The remains of a German hiker, who disappeared...",summarize:The remains of a German hiker who di...
4,"A UK-based doctor, Manish Shah, has been charg...",summarize:A GP who practised in east London ha...


In [21]:
class custom_dataset(Dataset):
  def __init__(self,dataframe,tokenizer,source_len,summ_len):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = summ_len
    self.text = self.data.text
    self.ctext = self.data.ctext
  
  def __len__(self):
    return len(self.text)
  
  def __getitem__(self, index):
    ctext = str(self.ctext[index])
    ctext = ' '.join(ctext.split())

    text = str(self.text[index])
    text = ' '.join(text.split())

    source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }


In [25]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
    

In [26]:
training_set = custom_dataset(train_dataset, tokenizer, 512, 150)
val_set = custom_dataset(val_dataset, tokenizer, 512, 150)

In [31]:
# Defining the parameters for creation of dataloaders
train_params = {
    'batch_size': 32,
    'shuffle': True,
    'num_workers': 0
    }

val_params = {
    'batch_size': 32,
    'shuffle': False,
    'num_workers': 0
    }


In [33]:
# Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)

In [34]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model = model.to(device)

In [35]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-4)